I was interested in the how severe weather has changed in the US since record keeping began, specifically how many deaths,injuries and the amount of property damage has been done by severe weather in the US. NOAA data for storms and weather anomalies goes back to 1950. Data is Publically available on Google Cloud BigQuery, but each year has separate table so manually querying the data will be time consuming. This python code will run the same query on each years data table and combine the results in a single place

Installing the google cloud library to access BigQuery through python


In [84]:
%%capture
%pip install --upgrade google-cloud-bigquery







Creating a google cloud bigquery client using a service account

In [85]:
from google.cloud import bigquery # to run queries on google clouds bigquery
from google.oauth2 import service_account # to access google cloud using a service account

credentials = service_account.Credentials.from_service_account_file(
    r"C:\Users\skicr\Downloads\alpine-tempo-392622-523114992507.json"
) 
google_cloud_project_id = 'alpine-tempo-392622' 
client = bigquery.Client(credentials=credentials,project=google_cloud_project_id)

Creating a list of identical SQL queries for each year of data being queried with the .format method

In [87]:
years = range(1950,2023)
queries = []
for year in years: 
    query=('''
            SELECT
                states.state,
                storm_data.deaths,
                storm_data.injuries,
                storm_data.damage
            FROM
              (
                SELECT
                  SUM(deaths_direct+deaths_indirect) AS deaths, 
                  SUM(injuries_direct+injuries_indirect)  AS injuries, 
                  SUM(damage_crops+damage_property) AS damage,
                  LPAD(state_fips_code,2,'0') as fips_code
                FROM
                  `bigquery-public-data.noaa_historic_severe_storms.storms_{year}`
                GROUP BY
                fips_code
              ) as storm_data
            RIGHT JOIN 
              `bigquery-public-data.geo_us_boundaries.states` AS states 
            ON 
              states.state_fips_code=storm_data.fips_code                               
            ORDER by 
              states.state
            ''').format(year=year)
    
    queries.append(query)


Installing pandas to make working with the data more convenient

In [88]:
%%capture
%pip install pandas 

Performing each query and combining the data

In [89]:
import pandas as pd

# to store the results of each query
results_list=[] 


for x, query in enumerate(queries):
   query = client.query(queries[x])           #running the query
   query_res = query.result().to_dataframe()  #converting the results to a dataframe
   query_res.insert(4,'year',years[x])        #adds a column to the results to show what year its from
   results_list.append(query_res)             #adding the dataframe to a list of all the results

#combining the list of tables into a single dataframe
results_df=pd.concat(results_list,ignore_index=True) 
results_df.tail()

,state,deaths,injuries,damage,year
4083,VT,2,2,5786000,2022
4084,WA,7,4,365580800,2022
4085,WI,30,8,22320000,2022
4086,WV,8,0,88300050,2022
4087,WY,3,2,25392380,2022


The NOAA's data for damage is not adjusted for inflation so the cpi library will be used to convert all dollar values to 2023 values and add a row for it in the results dataframe


In [90]:
%%capture
%pip install cpi

In [92]:
import cpi

#creating an inflation factor for each years dollars to convert it to current dollars(i.e. $1950*x=$2023)
inflation_factor = {} # creating a dictionary to store the factor for each year

for year in years:
   inflation_factor[year] = cpi.inflate(1,year) 

#adding a new column for inflation adjusted damage values using the apply method and inflation factor 
results_df['infl_adj_damage'] = results_df.apply(lambda row: row['damage']*inflation_factor[row['year']],axis=1)

results_df.head()



,state,deaths,injuries,damage,year,infl_adj_damage
0,AK,<NA>,<NA>,<NA>,1950,<NA>
1,AL,0,15,27500,1950,333942.427386
2,AR,2,49,860030,1950,10443654.757261
3,AS,<NA>,<NA>,<NA>,1950,<NA>
4,AZ,<NA>,<NA>,<NA>,1950,<NA>


Scraping the US Censes Bureau Website to create columns for deaths and injuries normalized by population for each state

Installing the US library to easily convert between state names and state codes

In [93]:
%%capture
%pip install us 

Defining a function to convert state names into state codes

In [94]:
%env DC_STATEHOOD = 1 #define before importing the us library to trear DC as a state, otherwise it returns NA
import us

def state_code_lookup(state_name):
    
    if type(state_name)==str:  #making sure the state name is a string
        state_name=state_name.lstrip('.')   #data from a table below had state names preceded by a period, this removes that
        if us.states.lookup(state_name) is not None:  #checking that the function will return a state
            return us.states.lookup(state_name).abbr  #returns the two letter state code
    else:
        return "Unknown" #to avoid returning any NA values if the string isnt a state

env: DC_STATEHOOD=1 #define before importing the us library to trear DC as a state, otherwise it returns NA


Using a table with US census data going back to 1910 to get the population for each state

In [95]:
import requests

#using the requests library to get the html from the website
census_url = 'https://www.census.gov/data/tables/time-series/dec/popchange-data-text.html'
census_html = requests.get(census_url).text

#using the read_html function to create a list of dataframes for each table on the website
pop_df_list=pd.read_html(census_html)

#selecting the first table in the list because there is only one on the webpage
popdf=pop_df_list[0]

 #preview the data
popdf.head(20)




,State or Region,2020 Census,2010 Census,2000 Census,1990 Census,1980 Census,1970 Census,1960 Census,1950 Census,1940 Census,1930 Census,1920 Census,1910 Census
0,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States
1,Resident Population,331449281,308745538,281421906,248709873,226545805,203211926,179323175,151325798,132165129,123202660,106021568,92228531
2,Percent Change,7.4%,9.7%,13.2%,9.8%,11.5%,13.3%,18.5%,14.5%,7.3%,16.2%,15.0%,21.0%
3,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast,Northeast
4,Resident Population,57609148,55317240,53594378,50809229,49135283,49040703,44677819,39477986,35976777,34427091,29662053,25868573
5,Percent Change,4.1%,3.2%,5.5%,3.4%,0.2%,9.8%,13.2%,9.7%,4.5%,16.1%,14.7%,22.9%
6,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest,Midwest
7,Resident Population,68985454,66927001,64392776,59668632,58865670,56571663,51619139,44460762,40143332,38594100,34019792,29888542
8,Percent Change,3.1%,3.9%,7.9%,1.4%,4.1%,9.6%,16.1%,10.8%,4.0%,13.4%,13.8%,13.5%
9,South,South,South,South,South,South,South,South,South,South,South,South,South


This table has an interesting layout and will need to be converted to a new layout to join it with the table of results

In [96]:
#setting the index as the state or region column for easier referencing
popdf.set_index('State or Region',inplace=True)

newrow=[] # list of dictionaries to add the extracted data needed

for col in popdf.columns: #iterating over each year
    for i,x in  enumerate(popdf[col]): #iterating over each row for the given year
        if x == str(us.states.lookup(x)): #checks if the row is a state
            if int(col[:4]) < 1950: #checks that the year is 1950 or later
                break
            else:
                #adding a dict containing state, population and year to the dict
                newrow.append(
                    
                    {'state':state_code_lookup(x),'population':int((popdf[col])[i+1]),'year':int(col[:4])}
                    
                    )

#combining all the data into a single data frame
consol_df=pd.DataFrame(newrow)


consol_df.head()


,state,population,year
0,AL,5024279,2020
1,AK,733391,2020
2,AZ,7151502,2020
3,AR,3011524,2020
4,CA,39538223,2020


Merging the two sets of data

In [98]:
merged_df=pd.DataFrame()
merged_df = results_df.merge(consol_df,on=['state','year'],how='left')
merged_df.tail()



,state,deaths,injuries,damage,year,infl_adj_damage,population
4083,VT,2,2,5786000,2022,5786000.0,NaN
4084,WA,7,4,365580800,2022,365580800.0,NaN
4085,WI,30,8,22320000,2022,22320000.0,NaN
4086,WV,8,0,88300050,2022,88300050.0,NaN
4087,WY,3,2,25392380,2022,25392380.0,NaN


Because there is only a census every 10 years, there is no population data for the years between censuses. However, an estimate can be made by interpolating the data between census years

In [99]:
%%capture
%pip install numpy
import numpy as np

In [102]:

merged_df.set_index(['state'],inplace=True) # setting the index to state for easier reference
states =[*merged_df.index.drop_duplicates()] #list of all states

merged_df.sort_values(['state','year'],inplace=True) #sorting data by state an then year
for state in states:
   merged_df.loc[state,'population'].interpolate(method='linear',inplace=True) #interpolating the population for each state
merged_df.head(11),merged_df.tail()


C:\Users\skicr\AppData\Local\Temp\ipykernel_8208\3023069214.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.loc[state,'population'].interpolate(method='linear',inplace=True) #interpolating the population for each state


(       deaths  injuries  damage  year infl_adj_damage  population
 state                                                            
 AK       <NA>      <NA>    <NA>  1950            <NA>    128643.0
 AK       <NA>      <NA>    <NA>  1951            <NA>    138395.4
 AK       <NA>      <NA>    <NA>  1952            <NA>    148147.8
 AK       <NA>      <NA>    <NA>  1953            <NA>    157900.2
 AK       <NA>      <NA>    <NA>  1954            <NA>    167652.6
 AK       <NA>      <NA>    <NA>  1955            <NA>    177405.0
 AK       <NA>      <NA>    <NA>  1956            <NA>    187157.4
 AK       <NA>      <NA>    <NA>  1957            <NA>    196909.8
 AK       <NA>      <NA>    <NA>  1958            <NA>    206662.2
 AK       <NA>      <NA>    <NA>  1959            <NA>    216414.6
 AK       <NA>      <NA>    <NA>  1960            <NA>    226167.0,
        deaths  injuries    damage  year  infl_adj_damage  population
 state                                                    

This worked well except for the last two years of data because there wasn't a final value to interpolate between. The census also publishes population estimates for each year which we can access to fix the last two years of population data. This will be done by importing the correct excel file with data from the census bureau's website using beautiful soup

In [104]:
%pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: dbapi 0.0.14 has a non-standard dependency specifier requests>. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dbapi or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [105]:
#url for the webpage with the file needed
url = 'https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html#v2022' 


html=requests.get(url).text #extracting the html
soup = bs(html,'html.parser') #creating a beautiful soup object for the webpage
filetracks = soup.find_all('a',filetrack=True,href=True) #selecting all links with a filetrack attribute

#the link we need is the second one on the webpage
filetrack=filetracks[1].get('href')
print(filetrack)


//www2.census.gov/programs-surveys/popest/tables/2020-2022/state/totals/NST-EST2022-POP.xlsx


The url has no scheme so it will not work to download the excel file. The urllib.parse library will be used to add the scheme

In [106]:

from urllib.parse import urlparse, urlunparse


In [107]:
filetrack=urlunparse(urlparse(filetrack,scheme='https'))
filetrack


'https://www2.census.gov/programs-surveys/popest/tables/2020-2022/state/totals/NST-EST2022-POP.xlsx'

Now we can import the file to a dataframe

In [108]:
%%capture
%pip install openpyxl



In [109]:
import openpyxl
more_pop_data = pd.read_excel(filetrack)
more_pop_data.head(10)


,table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
1,Geographic Area,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),NaN,NaN
2,NaN,NaN,2020,2021.0,2022.0
3,United States,331449520,331511512,332031554.0,333287557.0
4,Northeast,57609156,57448898,57259257.0,57040406.0
5,Midwest,68985537,68961043,68836505.0,68787595.0
6,South,126266262,126450613,127346029.0,128716192.0
7,West,78588565,78650958,78589763.0,78743364.0
8,.Alabama,5024356,5031362,5049846.0,5074296.0
9,.Alaska,733378,732923,734182.0,733583.0


Extracting the data needed in the form we need

In [110]:
#trimming to only the data needed
trmd_pop_data=more_pop_data.iloc[:,[0,3,4]] 

#renaming the columns
trmd_pop_data.rename(columns={'table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)':'State','Unnamed: 3':'2021','Unnamed: 4':'2022'},inplace=True)

#changing to state code instead state name to add to other table
trmd_pop_data['State']=trmd_pop_data.iloc[:,0].apply(state_code_lookup) 


trmd_pop_data.head(10)




C:\Users\skicr\AppData\Local\Temp\ipykernel_8208\146867293.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trmd_pop_data.rename(columns={'table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)':'State','Unnamed: 3':'2021','Unnamed: 4':'2022'},inplace=True)
C:\Users\skicr\AppData\Local\Temp\ipykernel_8208\146867293.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trmd_pop_data['State']=trmd_pop_data.iloc[:,0].apply(state_code_lookup)


,State,2021,2022
0,None,NaN,NaN
1,None,NaN,NaN
2,Unknown,2021.0,2022.0
3,None,332031554.0,333287557.0
4,None,57259257.0,57040406.0
5,None,68836505.0,68787595.0
6,None,127346029.0,128716192.0
7,None,78589763.0,78743364.0
8,AL,5049846.0,5074296.0
9,AK,734182.0,733583.0


This dataframe will need to be converted to the same format as the other dataframe to join them together

In [111]:
#setting state as the index to find population values based on state and year
trmd_pop_data.set_index('State',inplace=True)

row_update=[] # to store dictionaries with data
for state in states: #using a list of states from earlier
    if state in trmd_pop_data.index: #checking if the state is in the new dataframe
        for year in trmd_pop_data.columns: #iterating over each column excluding the first
            row_update.append({'state':state,'year':int(year),'population':trmd_pop_data.loc[state,year]}) #creating a dictionary and adding it to the list

In [112]:
#converting the list of dictionaries to a dataframe
row_update_df = pd.DataFrame(row_update)

#setting the indices of each data frame to the same columns so the update function works properly
merged_df.set_index('year',append=True,inplace=True)
row_update_df.set_index(['state','year'], inplace=True)

#updating the dataframe with the new population data
merged_df.update(row_update_df)
merged_df.tail()


deaths  injuries    damage  infl_adj_damage  population
state year                                                         
WY    2018       4        10  10382000  12099798.930336    574206.0
      2019       6         4   3212000   3676832.083612    575528.5
      2020       2        35     27000     30530.715464    576851.0
      2021       5         2   5314000     5739265.1216    579483.0
      2022       3         2  25392380       25392380.0    581381.0

Now that we have population data for each year we can calculate deaths and injuries per 100k residents

In [115]:
#all remaining NA values will be 0
merged_df.fillna(0,inplace=True)

In [116]:

#resetting the index so state and year are columns
final_df=merged_df.reset_index()

#converting all columns to integer data types
final_df = final_df.astype({'population':np.int64,'infl_adj_damage':np.int64})

#creating columns for deaths and injuries per 100k people
final_df['deaths/100k']=final_df.apply(lambda x: x['deaths']/x['population']*100000, axis=1)  
final_df['injuries/100k']=final_df.apply(lambda x: x['injuries']/x['population']*100000, axis=1)
final_df.head()

ZeroDivisionError: division by zero

Some rows still don't have population data

In [117]:
#selecting the states with population values of 0
final_df.loc[final_df['population']==0,['state']].drop_duplicates()

,state
219,AS
876,GU
1971,MP
3650,VI


These are small US territories where population data is hard to find so they will be removed

In [118]:
#creating a list of all row indices where the population is 0
rows_to_drop=final_df.loc[final_df['population']==0].index

#using that list to drop the rows
final_df.drop(index=rows_to_drop,axis=0,inplace=True)

In [119]:
final_df['deaths/100k']=final_df.apply(lambda x: x['deaths']/x['population']*100000, axis=1)  
final_df['injuries/100k']=final_df.apply(lambda x: x['injuries']/x['population']*100000, axis=1)
final_df.tail()

,state,year,deaths,injuries,damage,infl_adj_damage,population,deaths/100k,injuries/100k
4083,WY,2018,4,10,10382000,12099798,574206,0.696614,1.741535
4084,WY,2019,6,4,3212000,3676832,575528,1.042521,0.695014
4085,WY,2020,2,35,27000,30530,576851,0.346710,6.067425
4086,WY,2021,5,2,5314000,5739265,579483,0.862838,0.345135
4087,WY,2022,3,2,25392380,25392380,581381,0.516013,0.344008


To expand on this data in an analysis, another query can be run to return similar data for different kinds of weather events. The code above can be reused with a different query.

In [120]:
years = range(1950,2023)
queries = []
for year in years: 
  query=('''
            SELECT
                states.state,
                storm_data.event_type,
                storm_data.deaths,
                storm_data.injuries,
                storm_data.damage
            FROM
              (
                SELECT
                  event_type,
                  SUM(deaths_direct+deaths_indirect) AS deaths, 
                  SUM(injuries_direct+injuries_indirect)  AS injuries, 
                  SUM(damage_crops+damage_property) AS damage,
                  LPAD(state_fips_code,2,'0') as fips_code
                FROM
                  `bigquery-public-data.noaa_historic_severe_storms.storms_{year}`
                
                GROUP BY
                fips_code,event_type
              ) as storm_data
            right JOIN 
              `bigquery-public-data.geo_us_boundaries.states` AS states 
            ON 
              states.state_fips_code=storm_data.fips_code  
            WHERE 
              storm_data.deaths >0 or
              storm_data.injuries >0 or
              storm_data.damage >0                             
            ORDER by 
              states.state
            ''').format(year=year)
    
  queries.append(query)

# to store the results of each query
results_list=[] 


for x, query in enumerate(queries):
   query = client.query(queries[x])           #running the query
   query_res = query.result().to_dataframe()  #converting the results to a dataframe
   query_res.insert(4,'year',years[x])        #adds a column to the results to show what year its from
   results_list.append(query_res)             #adding the dataframe to a list of all the results

#combining the list of tables into a single dataframe
event_type_df=pd.concat(results_list,ignore_index=True) 
event_type_df.tail()
    

,state,event_type,deaths,injuries,year,damage
17519,WY,avalanche,2,1,2022,0
17520,WY,high wind,0,0,2022,27000
17521,WY,lightning,1,1,2022,0
17522,WY,flash flood,0,0,2022,336380
17523,WY,thunderstorm wind,0,0,2022,29000


In [122]:
#adding a new column for inflation adjusted damage values using the apply method and inflation factor 
event_type_df['infl_adj_damage'] = event_type_df.apply(lambda row: row['damage']*inflation_factor[row['year']],axis=1)
event_type_df=event_type_df.astype({'infl_adj_damage':np.int64})

event_type_df.head()


,state,event_type,deaths,injuries,year,damage,infl_adj_damage
0,AL,tornado,0,15,1950,27500,333942
1,AR,tornado,2,49,1950,860030,10443654
2,CO,tornado,0,1,1950,50000,607168
3,CT,tornado,0,3,1950,252500,3066198
4,FL,tornado,0,0,1950,82500,1001827


Examining the events types to see in consoslidation is possible

In [123]:
print(event_type_df['event_type'].drop_duplicates())

0                             tornado
1316                thunderstorm wind
1368                             hail
2113       tornadoes, tstm wind, hail
2317      thunderstorm winds/flooding
2418          thunderstorm wind/ tree
2419         thunderstorm wind/ trees
2454     thunderstorm winds lightning
2458        thunderstorm winds/ flood
2473                            flood
2474                         wildfire
2475                        avalanche
2476                        high wind
2477                        ice storm
2478                        lightning
2479                       heavy snow
2480                  cold/wind chill
2481                 storm surge/tide
2483                             heat
2488                      flash flood
2489                      rip current
2490                     winter storm
2491                   winter weather
2505                       dust devil
2506                       dust storm
2508                      strong wind
2514        

The categories don't need to be quite this specific so a dictionary will be used to consolidate

In [124]:
#keys are original event types and values are less specific terms
weather_dict={
    'funnel cloud':'tornado',
    'tornadoes, tstm wind, hail':'tornado',
    'thunderstorm wind':'thunderstorm',
    'thunderstorm winds/flooding':'thunderstorm', 
    'thunderstorm wind/ tree':'thunderstorm',
    'thunderstorm wind/ trees':'thunderstorm',
    'thunderstorm winds lightning':'thunderstorm',
    'thunderstorm winds/ flood':'thunderstorm',
    'hurricane (typhoon)':'hurricane',
    'high wind':'high winds',
    'strong wind':'high winds',
    'marine high wind':'high winds',
    'blizzard':'winter storm',
    'heavy snow':'winter storm',
    'lake-effect snow':'winter storm',
    'heat':'excessive heat',
    'cold/wind chill':'extreme cold',
    'extreme cold/wind chill':'extreme cold',
    'sleet':'winter weather',
    'freezing fog':'winter weather',
    'frost/freeze':'winter weather',
    'sneakerwave':'extreme waves',
    'high surf':'extreme waves',
    'seiche':'extreme waves',
    'lakeshore flood':'flood',
    'coastal flood':'flood'
    }

A function can be used to return the less specific term if the original term is a key in the dictionary

In [125]:

def weather_categories(w_type):
    if w_type in weather_dict:
        return weather_dict[w_type]
    else:
        return w_type

In [126]:

#applying the function to the event type column   
event_type_df['event_type']=event_type_df['event_type'].apply(weather_categories)

#checking to see if any slipped through
event_type_df['event_type'].drop_duplicates()

0                      tornado
1316              thunderstorm
1368                      hail
2473                     flood
2474                  wildfire
2475                 avalanche
2476                high winds
2477                 ice storm
2478                 lightning
2479              winter storm
2480              extreme cold
2481          storm surge/tide
2483            excessive heat
2488               flash flood
2489               rip current
2491            winter weather
2505                dust devil
2506                dust storm
2517                 dense fog
2518             extreme waves
2522                heavy rain
2575                waterspout
2581            tropical storm
2584                 hurricane
2666                   drought
2706               debris flow
7589                   tsunami
8562               dense smoke
8754       tropical depression
11465    astronomical low tide
Name: event_type, dtype: object

Converting the dataframes to an excel spreadsheet and saving it locally

In [127]:
%%capture
%pip install XlsxWriter


In [128]:
import xlsxwriter
with pd.ExcelWriter(r'c:\Users\skicr\Documents\Python Scripts\NOAA Storm Data\storm_data_by_year.xlsx') as writer:
    final_df.to_excel(writer,sheet_name='state_population_data',index=False,)
    event_type_df.to_excel(writer,sheet_name='event_type_data',index=False)  


All Done! This data will be used to create tableau visualizations. [Heat Maps by State](https://public.tableau.com/views/NOAASevereWeatherEventDeathsInjuriesandPropertyDamage1950-2022/DeathsInjuriesandPropertyDamageCausedbySevereWeather1950-2022?:language=en-US&:display_count=n&:origin=viz_share_link), Heat Maps Story, Event type Dashboard 

''